In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import json  
import codecs
import pandas as pd
import os
import shutil
from datetime import datetime, timedelta, date
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.firefox.options import Options
from os import remove
import git

def guardarRepositorio():
    repoLocal = git.Repo( 'C:/Users/limc_/Documents/GitHub/Datos' )
    print(repoLocal.git.status())
    repoLocal.git.add(".")
    try:
        repoLocal.git.commit(m='Update automatico via python')
    except:
        pass

    origin = repoLocal.remote(name='origin')
    origin.push()


def Chile():
    url_chile = "https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/covid19_chile.csv"
    pd.read_csv(url_chile).to_csv("Chile/covid19_chile.csv", index=False)

    url_comunas = "https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/covid19_comunas.csv"
    pd.read_csv(url_chile).to_csv("Chile/covid19_comunas.csv", index=False)

    url_old = "https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/old/covid19_chile.csv"
    pd.read_csv(url_chile).to_csv("Chile/covid19_chile_old.csv", index=False)
    
    #guardarRepositorio()
    
    return

def EarlyAlert():
    url  = 'https://services9.arcgis.com/Rha9bYQCF0JEy8bJ/ArcGIS/rest/services/Current_Coronavirus_Cases_and_Deaths/FeatureServer/0/1?f=pjson'
    response = requests.get(url)
    decoded_data=codecs.decode(response.content, 'utf-8-sig')
    d = json.loads(decoded_data)
    print(d["feature"]["attributes"])
    flag = True
    n = 1
    salida = []
    while flag:
        try:
            url  = 'https://services9.arcgis.com/Rha9bYQCF0JEy8bJ/ArcGIS/rest/services/Current_Coronavirus_Cases_and_Deaths/FeatureServer/0/'+str(n)+'?f=pjson'
            response = requests.get(url)
            decoded_data=codecs.decode(response.content, 'utf-8-sig')
            d = json.loads(decoded_data)
            print(d["feature"]["attributes"])
            salida.append(d["feature"]["attributes"])
            n += 1
        except:
            flag = False
    data = pd.DataFrame.from_dict(salida)
    #data.head()
    now = datetime.now()

    data.to_csv("EarlyAlert/Current_Coronavirus_Cases_and_Deaths.csv", index=False)
    data.to_csv("EarlyAlert/Current_Coronavirus_Cases_and_Deaths." + now.strftime("%d-%m-%Y_%H-%M-%S") + "csv", index=False)
    
    #guardarRepositorio()
    return
    
def ecdcEuropa():
    now = datetime.now()
    url= "https://opendata.ecdc.europa.eu/covid19/casedistribution/csv/"
    data = pd.read_csv(url)


    data.to_csv("ecdc.europa/Current_Coronavirus_Cases_and_Deaths.csv", index=False)
    data.to_csv("ecdc.europa/ecCurrent_Coronavirus_Cases_and_Deaths." + now.strftime("%d-%m-%Y_%H-%M-%S") + "csv", index=False)
    
    #guardarRepositorio()
    return

def johnsHopkinsCovid19():
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" 
    #01-22-2020.csv

    inicio = datetime(2020,1,22)
    fin    = datetime.now()

    lista_fechas = [inicio + timedelta(days=d) for d in range((fin - inicio).days + 1)] 
    for i in lista_fechas:
        #print(i.strftime("%m-%d-%Y.csv"))
        nombre = i.strftime("johns_Hopkins-covid19/diario/%m-%d-%Y.csv")
        try:
            ultimo = pd.read_csv(url + nombre)
            ultimo.to_csv(nombre,index=False)
        except:
            pass

    ultimo.to_csv("Johns_Hopkins-covid19/diario/ultimoRegistro.csv", index=False)
    
    pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv").to_csv("Johns_Hopkins-covid19/series/time_series_covid19_confirmed_US.csv", index=False)
    pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv").to_csv("Johns_Hopkins-covid19/series/time_series_covid19_confirmed_global.csv", index=False)
    pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv").to_csv("Johns_Hopkins-covid19/series/time_series_covid19_deaths_US.csv", index=False)
    pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv").to_csv("Johns_Hopkins-covid19/series/time_series_covid19_deaths_global.csv", index=False)
    pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv").to_csv("Johns_Hopkins-covid19/series/time_series_covid19_recovered_global.csv", index=False)
    
    #guardarRepositorio()
    return

def cambioFecha(texto):
    aux = str(texto)
    
    traductor ={
        "Jan" : 1,
        "Feb" : 2,
        "Mar" : 3
    }
    try:
        mes = traductor[aux[:3]]
    except:
        mes = 12
    #datetime.date(2019, 12, 4)
    #dia = int(aux[4:7])
    try:
        dia = int(aux[4:7])
    except:
        return None
    salida = datetime.date(2020,mes,dia)
    return salida
    #return aux[:3]
    
def worldometersInfo():
    url  = 'https://www.worldometers.info/coronavirus/'
    response = requests.get(url)
    #print(response.content)
    data = pd.read_html(response.content)
    data_hoy = data[0]     #.to_csv("worldometers.csv", index=False)
    data_ayer = data[1]     #.to_csv("worldometersAYER.csv", index=False)

    data_hoy["Primer caso"] = data_hoy["Reported1st case"].apply(cambioFecha)
    data_ayer["Primer caso"] = data_ayer["Reported1st case"].apply(cambioFecha)

    data_hoy.to_csv("worldometers.info/worldometers.csv", index=False)
    data_ayer.to_csv("worldometers.info/worldometersAYER.csv", index=False)
    
    #guardarRepositorio()
    return

def ourWorldInData():
    now = datetime.now()
    try:
        remove("C:/Users/limc_/Downloads/tests-vs-confirmed-cases-covid-19.csv")
        remove("C:/Users/limc_/Downloads/total-deaths-covid-19.csv")
        remove("C:/Users/limc_/Downloads/covid-19-tests-country.csv")

        remove("C:/Users/limc_/Downloads/total-daily-covid-deaths-per-million.csv")
        remove("C:/Users/limc_/Downloads/total-cases-covid-19.csv")
        remove("C:/Users/limc_/Downloads/total-and-daily-cases-covid-19.csv")
        remove("C:/Users/limc_/Downloads/daily-cases-covid-19.csv")
        remove("C:/Users/limc_/Downloads/physicians-per-1000-people.csv")
        remove("C:/Users/limc_/Downloads/hospital-beds-per-1000-people.csv")
        remove("C:/Users/limc_/Downloads/share-of-the-population-that-is-70-years-and-older.csv")
    except:
        pass
    download_folder = "C:/Users/limc_/Documents/GitHub/Datos"
    options = Options()
    options.set_preference("browser.download.dir", download_folder)
    #options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/csv") #, "application/json")
    #browser = webdriver.WebDriver(firefox_profile=profile)
    #driver = webdriver.WebDriver(firefox_profile=profile)
    driver = webdriver.Firefox(options=options)
    driver.set_page_load_timeout("60")

    driver.get("https://ourworldindata.org/grapher/tests-vs-confirmed-cases-covid-19")
    time.sleep(2)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[3]/div/nav/ul/li[2]/a").click()
    time.sleep(1)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[4]/div/a").click()
    time.sleep(1)

    driver.get("https://ourworldindata.org/grapher/covid-19-tests-country")
    time.sleep(2)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[3]/div/nav/ul/li[2]/a").click()
    time.sleep(1)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[4]/div/a").click()
    time.sleep(1)
    #*********************************************Con mapa sin mapa***********************************************
    driver.get("https://ourworldindata.org/grapher/total-deaths-covid-19")
    time.sleep(2)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[3]/div[2]/nav/ul/li[3]/a").click()
    time.sleep(1)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[4]/div/a").click()
    time.sleep(1) 

    driver.get("https://ourworldindata.org/grapher/total-daily-covid-deaths-per-million")
    time.sleep(2)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[3]/div[2]/nav/ul/li[3]/a").click()
    time.sleep(1)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[4]/div/a").click()
    time.sleep(1) 

    driver.get("https://ourworldindata.org/grapher/total-cases-covid-19")
    time.sleep(2)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[3]/div[2]/nav/ul/li[3]/a").click()
    time.sleep(1)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[4]/div/a").click()
    time.sleep(1)  

    driver.get("https://ourworldindata.org/grapher/total-and-daily-cases-covid-19")
    time.sleep(2)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[3]/div[2]/nav/ul/li[3]/a").click()
    time.sleep(1)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[4]/div/a").click()
    time.sleep(1)  

    driver.get("https://ourworldindata.org/grapher/daily-cases-covid-19")
    time.sleep(2)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[3]/div[2]/nav/ul/li[3]/a").click()
    time.sleep(1)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[4]/div/a").click()
    time.sleep(1)  

    driver.get("https://ourworldindata.org/grapher/physicians-per-1000-people")
    time.sleep(2)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[3]/div[2]/nav/ul/li[3]/a").click()
    time.sleep(1)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[4]/div/a").click()
    time.sleep(1)  

    driver.get("https://ourworldindata.org/grapher/hospital-beds-per-1000-people")
    time.sleep(2)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[3]/div[2]/nav/ul/li[3]/a").click()
    time.sleep(1)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[4]/div/a").click()
    time.sleep(1)  

    driver.get("https://ourworldindata.org/grapher/share-of-the-population-that-is-70-years-and-older")
    time.sleep(2)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[3]/div[2]/nav/ul/li[3]/a").click()
    time.sleep(1)
    driver.find_element_by_xpath("/html/body/main/figure/div/div[4]/div/a").click()
    time.sleep(1)  



    driver.close()

    shutil.copy('C:/Users/limc_/Downloads/tests-vs-confirmed-cases-covid-19.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/tests-vs-confirmed-cases-covid-19' + now.strftime("%d-%m-%Y_%H-%M-%S") + ".csv")
    shutil.copy('C:/Users/limc_/Downloads/tests-vs-confirmed-cases-covid-19.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/tests-vs-confirmed-cases-covid-19.csv')
    shutil.copy('C:/Users/limc_/Downloads/total-deaths-covid-19.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/total-deaths-covid-19' + now.strftime("%d-%m-%Y_%H-%M-%S") + ".csv")
    shutil.copy('C:/Users/limc_/Downloads/total-deaths-covid-19.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/total-deaths-covid-19.csv')
    shutil.copy('C:/Users/limc_/Downloads/covid-19-tests-country.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/covid-19-tests-country' + now.strftime("%d-%m-%Y_%H-%M-%S") + ".csv")
    shutil.copy('C:/Users/limc_/Downloads/covid-19-tests-country.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/covid-19-tests-country.csv')


    shutil.copy('C:/Users/limc_/Downloads/total-daily-covid-deaths-per-million.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/total-daily-covid-deaths-per-million' + now.strftime("%d-%m-%Y_%H-%M-%S") + ".csv")
    shutil.copy('C:/Users/limc_/Downloads/total-daily-covid-deaths-per-million.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/total-daily-covid-deaths-per-million.csv')

    shutil.copy('C:/Users/limc_/Downloads/total-cases-covid-19.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/total-cases-covid-19' + now.strftime("%d-%m-%Y_%H-%M-%S") + ".csv")
    shutil.copy('C:/Users/limc_/Downloads/total-cases-covid-19.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/total-cases-covid-19.csv')

    shutil.copy('C:/Users/limc_/Downloads/total-and-daily-cases-covid-19.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/total-and-daily-cases-covid-19' + now.strftime("%d-%m-%Y_%H-%M-%S") + ".csv")
    shutil.copy('C:/Users/limc_/Downloads/total-and-daily-cases-covid-19.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/total-and-daily-cases-covid-19.csv')


    shutil.copy('C:/Users/limc_/Downloads/daily-cases-covid-19.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/daily-cases-covid-19' + now.strftime("%d-%m-%Y_%H-%M-%S") + ".csv")
    shutil.copy('C:/Users/limc_/Downloads/daily-cases-covid-19.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/daily-cases-covid-19.csv')

    shutil.copy('C:/Users/limc_/Downloads/physicians-per-1000-people.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/physicians-per-1000-people' + now.strftime("%d-%m-%Y_%H-%M-%S") + ".csv")
    shutil.copy('C:/Users/limc_/Downloads/physicians-per-1000-people.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/physicians-per-1000-people.csv')

    shutil.copy('C:/Users/limc_/Downloads/hospital-beds-per-1000-people.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/hospital-beds-per-1000-people' + now.strftime("%d-%m-%Y_%H-%M-%S") + ".csv")
    shutil.copy('C:/Users/limc_/Downloads/hospital-beds-per-1000-people.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/hospital-beds-per-1000-people.csv')

    shutil.copy('C:/Users/limc_/Downloads/share-of-the-population-that-is-70-years-and-older.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/share-of-the-population-that-is-70-years-and-older' + now.strftime("%d-%m-%Y_%H-%M-%S") + ".csv")
    shutil.copy('C:/Users/limc_/Downloads/share-of-the-population-that-is-70-years-and-older.csv', 'C:/Users/limc_/Documents/GitHub/Datos/ourworldindata.org/share-of-the-population-that-is-70-years-and-older.csv')
    
    #guardarRepositorio()
    return




ModuleNotFoundError: No module named 'git'